# 商品検索AIエージェントのサンプルコード

## ADKのインストール

In [51]:
%pip uninstall -y google-adk
%pip install --upgrade --no-cache-dir --quiet google-adk

Found existing installation: google-adk 0.2.0
Uninstalling google-adk-0.2.0:
  Successfully uninstalled google-adk-0.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.4 MB/s eta 0:00:00


### ランタイム再起動

In [53]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## ライブラリの初期化

In [1]:
import copy, json, os, re, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.agent_tool import AgentTool

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

## エージェント実行用 LocalApp の定義

In [2]:
class LocalApp:
    def __init__(self, agent):
        self._agent = agent
        self._user_id = 'local_app'
        self._runner = Runner(
            app_name=self._agent.name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._agent.name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid1().hex,
        )

    async def _stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        agent_name = None
        async for event in async_events:
            if DEBUG:
                print(f'----\n{event}\n----')
            if (event.content and event.content.parts):
                response = ''
                for p in event.content.parts:
                    if p.text:
                        response += f'[{event.author}]\n\n{p.text}\n'
                if response:
                    #### Temporary fix for wrong agent routing message
                    pattern = 'transfer_to_agent\(agent_name=["\']([^"]+)["\']\)'
                    matched = re.search(pattern, response)
                    if (not agent_name) and matched:
                        agent_name = matched.group(1)
                    else:
                        print(response)
                        result.append(response)
                    ####
        return result, agent_name

    async def stream(self, query):
        result, agent_name = await self._stream(query)
        #### Temporary fix for wrong agent routing message
        if agent_name:
            if DEBUG:
                print(f'----\nForce transferring to {agent_name}\n----')
            result, _ = await self._stream(f'Please transfer to {agent_name}')
        ####
        return result

## 基本のエージェント定義

In [3]:
instruction = f'''
    Your role is a shop search agent on an e-commerce site with millions of
    items. Your responsibility is to search items based on user queries.
'''

item_search_agent = Agent(
    model='gemini-2.0-flash-001',
    name='item_search_agent',
    description=(
        'Item search agent for an e-commerce site'
    ),
    instruction=instruction,
)

In [4]:
client = LocalApp(item_search_agent)
DEBUG = False

query = f'''
こんにちは！　ここはどんなサイトですか？
'''
_ = await client.stream(query)

[item_search_agent]

こんにちは！ここは数百万点の商品を取り扱うeコマースサイトです。何かお探しのものはありますか？




## 商品検索APIの呼び出し関数

In [5]:
import requests

def call_query_api(api_url, query, rows=None, dataset_id=None, use_dense=True, use_sparse=False):
    """
    Calls the Flask API endpoint for querying.

    Args:
        api_url (str): The URL of the search endpoint.
        query (str): The query string.
        rows (int, optional): The number of result rows to return. Defaults to None.
        dataset_id (str, optional): The ID of the dataset to query. Defaults to None.
        use_dense (bool, optional): Whether to use dense embeddings. Defaults to True.
        use_sparse (bool, optional): Whether to use sparse embeddings. Defaults to False.

    Returns:
        dict: The JSON response from the API.
    """
    headers = {'Content-Type': 'application/json'}
    payload = {
        "query": query,
        "rows": rows,
        "dataset_id": dataset_id,
        "use_dense": use_dense,
        "use_sparse": use_sparse,
        "rrf_alpha": None,
        "use_rerank": None,
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error calling the API: {e}")
        return None

## 商品検索Toolの定義

In [6]:
from typing import Dict

def find_shopping_items(queries: list[str]) -> Dict[str, str]:
    """
    Find shopping items from the e-commerce site with the specified list of
    queries.

    Args:
        queries: the list of queries to run.
    Returns:
        A dict with the following one property:
            - "status": returns the following status:
                - "success": successful execution
            - "items": items found in the e-commerce site.
    """
    api_endpoint = "https://www.ac0.cloudadvocacyorg.joonix.net/api/query"

    items = []
    for query in queries:
        result = call_query_api(
            api_url=api_endpoint,
            query=query,
            dataset_id="mercari3m_mm", # Mercari 3M items Multimodal Embeddings
            rows=3,
        )
        items.extend(result["items"])

    print("-----")
    print(f"User queries: {queries}")
    print(f"Found: {len(items)} items")
    print("-----")

    return items

In [7]:
find_shopping_items(["Cups with dancing people", "Cups with dancing animals"])

-----
User queries: ['Cups with dancing people', 'Cups with dancing animals']
Found: 6 items
-----


[{'dense_dist': 0.20211432874202728,
  'description': 'Vintage hardanger dancer bergquist figgjo coffee mugs',
  'id': 'm10172014563',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m10172014563_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Vintage Berquist Figgjo coffee mugs',
  'rerank_score': 0.0,
  'sparse_dist': None,
  'url': 'https://www.mercari.com/us/item/m10172014563'},
 {'dense_dist': 0.1960698664188385,
  'description': 'Pottery Barn Christmas Reindeer Mugs - Dasher, Dancer, Prancer & Vixen Set of 4',
  'id': 'm81366738028',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m81366738028_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Pottery barn reindeer mugs',
  'rerank_score': 0.0,
  'sparse_dist': None,
  'url': 'https://www.mercari.com/us/item/m81366738028'},
 {'dense_dist': 0.18155741691589355,
  'description': 'VINTAGE CERAMIC MUG \nHOLLAND THEME \nCRAZING INSIDE NO CHIPS OR CRACKS\nBC MARKED ON BOTTOM \nSMOKE-FREE HOME\n\n\n\n\n3/23-2\n1lb 4oz\nSHD'

## 商品検索Toolに対応したエージェントの定義

In [8]:
instruction = f'''
    Your role is a shop search agent on an e-commerce site with millions of
    items. Your responsibility is to search items based on the queries you
    recieve.

    To find items use `find_shopping_items` tool by passing a list of queries.
    If the user requests in non-English languate, translate the query into
    English. When you recieved a list of items from the tool, answer to the
    user with item's name, description and img_url, by translating them back to
    the user's language.
'''

item_search_agent = Agent(
    model='gemini-2.0-flash-001',
    name='item_search_agent',
    description=(
        'Item search agent for an e-commerce site'
    ),
    instruction=instruction,
    tools=[find_shopping_items],
)

In [9]:
client = LocalApp(item_search_agent)
DEBUG = False

query = f'''
踊っている人が描かれたコップを探してください。
'''
_ = await client.stream(query)

-----
User queries: ['A cup with a picture of a dancing person']
Found: 3 items
-----
[item_search_agent]

 踊っている人が描かれたコップは以下の通りです。

1. 商品名：背の高いティガーマグカップ（16オンス）

説明：ディズニーブランドのティガーがマグカップの中や外で飛び跳ねています。 16オンス

画像のURL：https://u-mercari-images.mercdn.net/photos/m18336396506_1.jpg?w=200&h=200&fitcrop&sharpen

2. 商品名：ジーン・ムーア「ファンタジー」ティファニーカップ

説明：一番かわいい小さなカップ。 ダブルハンドルで、カラフルなモチーフが付いています。 欠けやひび割れはありません。

画像のURL：https://u-mercari-images.mercdn.net/photos/m66490364828_1.jpg?w=200&h=200&fitcrop&sharpen

3. 商品名：KOKOPELLIアーティストサイン入りコーヒーマグ

説明：HTF KOKOPELLIコーヒーマグ、アーティスト「アースメディスン」。 リムに小さな傷があり、ハンドルの付け根に欠けがあります。 使用によりマグカップの口の周りが若干色あせています。 写真をよくご確認ください。

画像のURL：https://u-mercari-images.mercdn.net/photos/m91064351259_1.jpg?w=200&h=200&fitcrop&sharpen




## Google検索によるクエリ生成エージェントの定義

In [10]:
from google.adk.tools import google_search

instruction = f'''
    Your role is a market researcher for an e-commerce site with millions of
    items.

    When you recieved a search request from an user, use Google Search tool to
    research on what kind of items people are purchasing for the user's intent.

    Then, generate 5 queries finding those items on the e-commerce site and
    return them.
'''

market_research_agent = Agent(
    model='gemini-2.0-flash-001',
    name='market_research_agent',
    description=('''
        A market researcher for an e-commerce site. Receives a search request
        from a user, and returns a list of 5 generated queries in English.
    '''),
    instruction=instruction,
    tools=[google_search],
)

In [11]:
client = LocalApp(market_research_agent)
DEBUG = False

query = f'''
birthday present for 10 years old boy
'''
_ = await client.stream(query)

[market_research_agent]

Okay, here are 5 potential search queries for an e-commerce site, designed to find a birthday present for a 10-year-old boy:

1.  "STEM kits for 10 year old boys"
2.  "LEGO sets for 10 year old boys age 10+"
3.  "Outdoor games for 10 year old active boys"
4.  "Remote control toys for 10 year old boys"
5.  "Board games age 10+ for boys family"




## 商品検索コンシェルジュエージェントの定義

In [15]:
instruction = f'''
    Your role is a shopper's concierge for an e-commerce site with millions of
    items. Follow the following steps. All communication with the user should
    be done in the user's language.

    1. Market research: When you recieved a search request from an user,
    translate the request to English, and pass the translated request to
    `market_research_agent` tool, and receive 5 generated queries. Share
    the queries with the user, and ask if they want to continue.

    2. Find items: When use requested finding items with the queries, pass
    the list of queries to `find_shopping_items` to
    find items. When you recieved a list of items from the tool, answer to the
    user with item's name, description and the image url, by
    translating them back to the user's language.
'''

shop_concierge_agent = Agent(
    model='gemini-2.0-flash-001',
    name='shop_concierge_agent',
    description=(
        'A shopper\'s concierge for an e-commerce site'
    ),
    instruction=instruction,
    tools=[
        AgentTool(agent=market_research_agent),
        find_shopping_items,
    ],
)

In [16]:
client = LocalApp(shop_concierge_agent)
DEBUG = False

query = f'''
10歳の息子の誕生日プレゼントを探してください
'''
_ = await client.stream(query)

[shop_concierge_agent]

10歳の息子さんへの誕生日プレゼントをお探しなのですね。承知いたしました。

まず、いくつか質問を作成して、プレゼントの候補を絞り込みたいと思います。
以下のクエリで検索をかけてもよろしいでしょうか？

1. "best birthday gifts for 10 year old boys"
2. "top toys for 10 year old boys 2024"
3. "popular video games for 10 year olds"
4. "creative gifts for 10 year old boys"
5. "educational toys for 10 year old boys"

これらのクエリで検索をかけてもよろしいでしょうか？




In [17]:
query = f'''
はい、お願いします！
'''
_ = await client.stream(query)

-----
User queries: ['best birthday gifts for 10 year old boys', 'top toys for 10 year old boys 2024', 'popular video games for 10 year olds', 'creative gifts for 10 year old boys', 'educational toys for 10 year old boys']
Found: 15 items
-----
[shop_concierge_agent]

10歳のお子様向けの誕生日プレゼントとして、いくつか商品が見つかりました。

1. **Gel blaster (blue)**: 水のビーズを使用するおもちゃの銃です。12歳以上推奨とのことです。
   * 説明: Electric Gel Blaster Toys, Automatic Splat Ball Blaster MP5 with 31000 Water Beads and Goggles。Ages 12+ recommended。
   * 画像URL: https://u-mercari-images.mercdn.net/photos/m49651033077_1.jpg?w=200&h=200&fitcrop&sharpen

2. **Flashing Cube Electronic Memory Game | 4-in-1 STEM Game**: 光るキューブを使った電子メモリーゲームです。6歳から106歳まで楽しめます。
   * 説明: Geared for kids and adults ages 6 to 106, FlashDash offers fun and adventure packed into a compact light-up cube that’s easy to take along with you for entertainment on-the-go.
   * 画像URL: https://u-mercari-images.mercdn.net/photos/m59901702555_1.jpg?w=200&h=200&fitcrop&sharpen

3. **Kids 